In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/webis-clickbait-22/test.jsonl
/kaggle/input/webis-clickbait-22/validation.jsonl
/kaggle/input/webis-clickbait-22/train.jsonl
/kaggle/input/logits-roberta-deberta-final/roberta_val_without_para.npy
/kaggle/input/logits-roberta-deberta-final/deberta_train_without_para.npy
/kaggle/input/logits-roberta-deberta-final/roberta_test_without_para.npy
/kaggle/input/logits-roberta-deberta-final/deberta_test_with_para.npy
/kaggle/input/logits-roberta-deberta-final/roberta_train_with_para.npy
/kaggle/input/logits-roberta-deberta-final/deberta_test_without_para.npy
/kaggle/input/logits-roberta-deberta-final/deberta_val_without_para.npy
/kaggle/input/logits-roberta-deberta-final/roberta_val_with_para.npy
/kaggle/input/logits-roberta-deberta-final/roberta_train_without_para.npy
/kaggle/input/logits-roberta-deberta-final/roberta_test_with_para.npy
/kaggle/input/logits-roberta-deberta-final/deberta_train_with_para.npy
/kaggle/input/logits-roberta-deberta-final/deberta_val_with_para.npy
/ka

In [2]:
! pip install accelarate -U
! pip install datasets

ERROR: Could not find a version that satisfies the requirement accelarate (from versions: none)
ERROR: No matching distribution found for accelarate


In [3]:
import wandb
api_key = "9963cf6219e451d47251ea34645181ada1b2526b"
wandb.login(key=api_key)
wandb.init()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: raghav21274 (ragha). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240425_124625-t336we2q
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run unique-terrain-116
wandb: ⭐️ View project at https://wandb.ai/ragha/uncategorized
wandb: 🚀 View run at https://wandb.ai/ragha/uncategorized/runs/t336we2q


In [4]:
from sklearnex import patch_sklearn
patch_sklearn()

import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import matplotlib.pyplot as plt

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

PATH="/kaggle/input/webis-clickbait-22/"
PATH_LOGITS="/kaggle/input/logits-roberta-deberta-final/"
OUTPATH="/kaggle/working/"

BATCH=8
sep='[SEP]'

roberta = "roberta-base"
deberta = "microsoft/deberta-base"

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
2024-04-25 12:46:54.789543: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 12:46:54.789673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 12:46:54.955295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using CPU


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer

class Dataset(torch.utils.data.Dataset):
    def __init__(self,encodings,labels=None):
        self.labels = labels
        self.encodings = encodings
    
    def __getitem__(self,idx):
        item = {key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
        if self.labels is not None:
            item["label"]  = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [6]:
data_train=pd.read_json("/kaggle/input/webis-clickbait-22/train.jsonl",lines=True,encoding='utf-8')
data_test=pd.read_json("/kaggle/input/webis-clickbait-22/test.jsonl",lines=True,encoding='utf-8')
data_val=pd.read_json("/kaggle/input/webis-clickbait-22/validation.jsonl",lines=True,encoding='utf-8')


In [7]:
label={
    'multi':0,
    'passage':1,
    'phrase':2
}

In [8]:

def get_split_data(data,if_train,option):
    global label_encoder
#     data=pd.read_json(file_path,lines=True,encoding='utf-8')

    y = data["tags"]
    Y = [i[0] for i in y]
    print("len of Y: ",len(Y))
    if if_train:
        Y=[label[i] for i in Y]
    else:
        Y =[label[i] for i in Y]

    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords']]
    dic=new_data.to_dict(orient='records')

    desccount=0
    keycount=0
    for i in dic:
        # print(i)
        i['postText']='postText: '+i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']='targetParagraphs: '+targetparah
        i['targetTitle']='targetTitle: '+i['targetTitle']
        targetDescription=i['targetDescription']
        if(targetDescription is None or len(targetDescription)==0):
            targetDescription='none'
            desccount+=1
        i['targetDescription']='targetDescription: '+targetDescription
        targetkey=i['targetKeywords']
        if(targetkey is None or len(targetkey)==0):
            targetkey='none'
            keycount+=1
        i['targetKeywords']='targetKeywords: '+targetkey
        if option==1:
            i['text']=i['postText']+" "+i['targetTitle']+" "+i['targetKeywords']+" "+sep+i['targetDescription']
        else:
            i['text']=i['targetParagraphs']+sep+i['postText']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    print("desccount: ",desccount)
    print("keycount: ",keycount)
    X = [i['text'] for i in dic]
    X=np.array(X)
#     return X[0:100],Y[0:100]
    return X,Y

In [9]:
x_train_without_para,y_train = get_split_data(data_train,True,1)
x_test_without_para,y_test = get_split_data(data_test,False,1)
x_val_without_para,y_val= get_split_data(data_val,False,1)

len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [10]:
x_train_with_para,y_train = get_split_data(data_train,True,0)
x_test_with_para,y_test = get_split_data(data_test,False,0)
x_val_with_para,y_val= get_split_data(data_val,False,0)

len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [11]:
logits_train_roberta_without_para = np.load(PATH_LOGITS+"roberta_train_without_para.npy")
logits_train_roberta_with_para = np.load(PATH_LOGITS+"roberta_train_with_para.npy")
logits_train_deberta_without_para = np.load(PATH_LOGITS+"deberta_train_without_para.npy")
logits_train_deberta_with_para = np.load(PATH_LOGITS+"deberta_train_with_para.npy")

In [12]:
logits_val_roberta_without_para = np.load(PATH_LOGITS+"roberta_val_without_para.npy")
logits_val_roberta_with_para = np.load(PATH_LOGITS+"roberta_val_with_para.npy")
logits_val_deberta_without_para = np.load(PATH_LOGITS+"deberta_val_without_para.npy")
logits_val_deberta_with_para = np.load(PATH_LOGITS+"deberta_val_with_para.npy")

In [13]:
logits_test_roberta_without_para = np.load(PATH_LOGITS+"roberta_test_without_para.npy")
logits_test_roberta_with_para = np.load(PATH_LOGITS+"roberta_test_with_para.npy")
logits_test_deberta_without_para = np.load(PATH_LOGITS+"deberta_test_without_para.npy")
logits_test_deberta_with_para = np.load(PATH_LOGITS+"deberta_test_with_para.npy")

In [14]:
combined_train = np.concatenate([logits_train_roberta_without_para,logits_train_roberta_with_para,logits_train_deberta_without_para,logits_train_deberta_with_para],axis=2)
combined_val = np.concatenate([logits_val_roberta_without_para,logits_val_roberta_with_para,logits_val_deberta_without_para,logits_val_deberta_with_para],axis=2)
combined_test = np.concatenate([logits_test_roberta_without_para,logits_test_roberta_with_para,logits_test_deberta_without_para,logits_test_deberta_with_para],axis=2)

In [15]:
combined_train = np.squeeze(combined_train,axis=1)
combined_val = np.squeeze(combined_val,axis=1)
combined_test = np.squeeze(combined_test,axis=1)

In [16]:
standard_scaler = StandardScaler()
combined_train = standard_scaler.fit_transform(combined_train)
combined_val = standard_scaler.transform(combined_val)
combined_test = standard_scaler.transform(combined_test)

INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


In [17]:
param_grid = {
    'C': [0.2,0.4,0.6,0.8,1,1.2],
    'gamma': [0.1, 0.01, 0.001],
    'kernel': ['rbf', 'poly', 'sigmoid','linear']
}

grid = GridSearchCV(SVC(), param_grid, cv=5,verbose=4,scoring="balanced_accuracy")

grid.fit(combined_train, y_train)

grid.best_params_

INFO:sklearnex: sklearn.svm.SVC.fit: running accelerated version on CPU


Fitting 5 folds for each of 72 candidates, totalling 360 fits


INFO:sklearnex: sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.svm.SVC.fit: running accelerated version on CPU


[CV 1/5] END ......C=0.2, gamma=0.1, kernel=rbf;, score=0.895 total time=   0.3s


INFO:sklearnex: sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.svm.SVC.fit: running accelerated version on CPU


...

INFO:sklearnex: sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.svm.SVC.fit: running accelerated version on CPU


[CV 4/5] END .C=1.2, gamma=0.001, kernel=linear;, score=0.888 total time=   0.3s


INFO:sklearnex: sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.svm.SVC.fit: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


[CV 5/5] END .C=1.2, gamma=0.001, kernel=linear;, score=0.930 total time=   0.3s


{'C': 0.6, 'gamma': 0.1, 'kernel': 'linear'}

In [18]:
C = grid.best_params_['C']
gamma = grid.best_params_['gamma']
kernel = grid.best_params_['kernel']
model = SVC(C=C,gamma=gamma,kernel=kernel)
model.fit(combined_train,y_train)

INFO:sklearnex: sklearn.svm.SVC.fit: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


SVC(C=0.6, gamma=0.1, kernel='linear')

In [19]:
y_pred_train=model.predict(combined_train)
y_pred_val=model.predict(combined_val)

print("Training Metrics")
print(balanced_accuracy_score(y_train,y_pred_train))
print(f1_score(y_train,y_pred_train,average='macro'))
print(classification_report(y_train,y_pred_train))

print("Validation Metrics")
print(balanced_accuracy_score(y_val,y_pred_val))
print(f1_score(y_val,y_pred_val,average='macro'))
print(classification_report(y_val,y_pred_val))

INFO:sklearnex: sklearn.svm.SVC.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.svm.SVC.predict: running accelerated version on CPU


Training Metrics
0.9018150913272515
0.9076546587737853
              precision    recall  f1-score   support

           0       0.94      0.87      0.90       559
           1       0.89      0.94      0.91      1274
           2       0.92      0.90      0.91      1367

    accuracy                           0.91      3200
   macro avg       0.91      0.90      0.91      3200
weighted avg       0.91      0.91      0.91      3200

Validation Metrics
0.7244869558302395
0.7292373918426698
              precision    recall  f1-score   support

           0       0.73      0.67      0.70       143
           1       0.71      0.79      0.75       322
           2       0.76      0.72      0.74       335

    accuracy                           0.74       800
   macro avg       0.74      0.72      0.73       800
weighted avg       0.74      0.74      0.74       800



In [20]:
with open('svm_ensemble.pkl', 'wb') as file:
    pickle.dump(model, file)

In [21]:
with open("svm_ensemble.pkl","rb") as f:
    loaded_model = pickle.load(f)

In [22]:
y_pred = loaded_model.predict(combined_test)
print('Testing accuracy {:.4f}'.format(accuracy_score(y_test, y_pred)))
print("Testing data metrics:")
print("Balanced Accuracy {:.4f} : ".format(balanced_accuracy_score(y_test,y_pred)))
print("Balacned F1 Score {:.4f} : ".format(f1_score(y_test,y_pred,  average='macro')))
print(classification_report(y_test,y_pred))

INFO:sklearnex: sklearn.svm.SVC.predict: running accelerated version on CPU


Testing accuracy 0.7250
Testing data metrics:
Balanced Accuracy 0.6987 : 
Balacned F1 Score 0.7062 : 
              precision    recall  f1-score   support

           0       0.69      0.59      0.63       174
           1       0.72      0.78      0.75       403
           2       0.74      0.73      0.74       423

    accuracy                           0.73      1000
   macro avg       0.72      0.70      0.71      1000
weighted avg       0.72      0.72      0.72      1000

